In [1]:
import pandas as pd
import numpy as np
import difflib #for close match like if he type wrong movie name
from sklearn.feature_extraction.text import TfidfVectorizer #convert text data into numberized form
from sklearn.metrics.pairwise import cosine_similarity #similarity checking algorithms

In [2]:
#data preprossing
dataset = pd.read_csv("E:\Data_Scientist\Python\Projects\Movie Recommedation System\movies.csv")
dataset.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
dataset.shape

(4803, 24)

In [4]:
#selecting what exact features u need
selectedFeatures = ['genres','keywords','tagline','cast','director']


#now check for missing values
dataset.isnull()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4799,False,False,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4800,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4801,False,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
dataset.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [6]:
#since that not much data missing from the features we selected so replacing them with null string since data is in text form

for i in selectedFeatures:
    dataset[i] = dataset[i].fillna('')

In [7]:
dataset.isnull().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [8]:
#combine all features

combinedFeatures = dataset['genres']+' '+dataset['keywords']+' '+dataset['tagline']+' '+dataset['cast']+' '+dataset['director']
combinedFeatures

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [9]:
#Converting text to features vectors only convert moost repested words ionto text

vectorizer = TfidfVectorizer()

featureVectors = vectorizer.fit_transform(combinedFeatures)
print(featureVectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [10]:
#applying algorithm
similarity = cosine_similarity(featureVectors)



In [11]:
#finally taking input from user
movieName = input("Any specific movie? ")

Any specific movie? Thor


In [12]:
#first compare all title with given name
allTitle = dataset['title'].tolist()
allTitle

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [18]:
#find close match using difflib to get close match
closeMatch = difflib.get_close_matches(movieName,allTitle,1) 
movieMatch = closeMatch[0]
print(movieMatch)

Thor


In [19]:
 #finding movie from given title
movieIndex = dataset[dataset.title==movieMatch]['index'].values[0]
print(movieIndex)

129


In [20]:
#find similar movies
similarMovies = list(enumerate(similarity[movieIndex]))#gives similarity confidence value
print(similarMovies)


[(0, 0.026944697335545255), (1, 0.15873396251727565), (2, 0.029587636800368036), (3, 0.026270340443845273), (4, 0.014091231284482253), (5, 0.042879250964057675), (6, 0.02491203147414628), (7, 0.2173754078053571), (8, 0.0392308668530781), (9, 0.05867811067491897), (10, 0.03030658132545716), (11, 0.012855393395249944), (12, 0.162544315517465), (13, 0.06393431075479926), (14, 0.05911418196211956), (15, 0.035408236669037536), (16, 0.26928516313639983), (17, 0.026318337548389012), (18, 0.005966465146181676), (19, 0.023366413601787386), (20, 0.09845275954463244), (21, 0.012144830426719468), (22, 0.01869533136700644), (23, 0.07774779211675045), (24, 0.013652128790913237), (25, 0.0), (26, 0.15859616283733802), (27, 0.09078118484639329), (28, 0.03274592475638117), (29, 0.015140715783620869), (30, 0.1413101509249316), (31, 0.0848991054611594), (32, 0.030691337370221845), (33, 0.1193153176718858), (34, 0.0), (35, 0.015238144451509856), (36, 0.013730784182362676), (37, 0.021765077721506338), (38, 

In [21]:
#simply sort based on similarity score

sortMovies = sorted(similarMovies, key = lambda x:x[1], reverse=True)
print(sortMovies)

[(129, 1.0), (126, 0.38805680765984835), (16, 0.26928516313639983), (326, 0.23491579692685097), (7, 0.2173754078053571), (737, 0.21507587600903866), (85, 0.21463221124856754), (2373, 0.1842415819272965), (3177, 0.18389041064588307), (3250, 0.17911895415003345), (12, 0.162544315517465), (128, 0.15884312986645496), (1, 0.15873396251727565), (26, 0.15859616283733802), (2401, 0.1578234934611817), (2035, 0.14690850526099808), (92, 0.14557998542213169), (30, 0.1413101509249316), (840, 0.1362875036742051), (594, 0.13517647521244153), (788, 0.13497208252963788), (38, 0.13412409894821842), (3435, 0.1331023421202746), (79, 0.13177598006322738), (354, 0.12823280037664625), (107, 0.1271290991238438), (2040, 0.12645964617939354), (46, 0.12632927428141277), (517, 0.12530868103754308), (2474, 0.12381228600956372), (2377, 0.12284253170723058), (886, 0.12283684348113257), (174, 0.1215255784736921), (511, 0.12120029438300144), (2975, 0.12056566255097574), (169, 0.11934058347622052), (33, 0.1193153176718

In [25]:
#finally printing first 30 movies
print("You may also like : ")

i=1
for movie in sortMovies:
    ind = movie[0]
    titles = dataset[dataset.index==ind]['title'].values[0]
    
    #just recommed first 30 movie
    if(i<=30):  
        print(i,'.',titles)
        i+=1
            

You may also like : 
1 . Thor
2 . Thor: The Dark World
3 . The Avengers
4 . Cinderella
5 . Avengers: Age of Ultron
6 . Jack Ryan: Shadow Recruit
7 . Captain America: The Winter Soldier
8 . Hamlet
9 . Henry V
10 . Much Ado About Nothing
11 . Pirates of the Caribbean: Dead Man's Chest
12 . Angels & Demons
13 . Pirates of the Caribbean: At World's End
14 . Captain America: Civil War
15 . City of Ghosts
16 . Our Kind of Traitor
17 . How to Train Your Dragon
18 . Spider-Man 2
19 . Ronin
20 . Bad Company
21 . Deadpool
22 . The Amazing Spider-Man 2
23 . Aberdeen
24 . Iron Man 2
25 . The Girl with the Dragon Tattoo
26 . Warcraft
27 . The Glass House
28 . X-Men: Days of Future Past
29 . Exorcist: The Beginning
30 . Arn: The Knight Templar
